DATOS LIMPIOS Y PROCESADOS

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Función para imputar la media o mediana agrupada por COMP_S_NAME
def impute_grouped_data(df, column, method='mean'):
    if method == 'median':
        return df.groupby('COMP_S_NAME')[column].transform(lambda x: x.fillna(x.median()))
    elif method == 'linear':
        return df[column].interpolate(method='linear')
    else:
        return df.groupby('COMP_S_NAME')[column].transform(lambda x: x.fillna(x.mean()))

# Cargar las bases de datos
produccion_df = pd.read_csv("produccion(1).csv")
test_df = pd.read_csv("TEST21.csv")


# Eliminar duplicados
produccion_df = produccion_df.drop_duplicates()
test_df = test_df.drop_duplicates()

# Eliminar columnas innecesarias de la tabla de producción
produccion_df = produccion_df.drop(columns=['OPERACION', 'ACTIVO', 'BLOQUE_N', 'ROUTE_NAME', 'PAD', 'ANIO', 'MES'])

# Eliminar filas con valores faltantes en columnas críticas
critical_columns_produccion = ['DIAS', 'POZO', 'COMP_S_NAME']
critical_columns_test = ['BSW', 'CASING_PRESS', 'PBHP', 'TUBING_PRESS', 'PIP', 'FLAP', 'SBHP', 'TUBING_TMP', 'AMPS_A', 'PUMP_TMP', 'MOTOR_HZ', 'POWER_KW', 'STAGE_COUNT', 'VOLTAGE', 'INTAKE_DEPTH', 'PUMP_TYPE', 'SALINIDAD', 'INJECTION_VOL', 'RETURN_VOL']

produccion_df = produccion_df.dropna(subset=critical_columns_produccion)
test_df = test_df.dropna(subset=critical_columns_test)

# Imputar valores faltantes en columnas seleccionadas con el método recomendado
produccion_df['PC_BPPD'] = impute_grouped_data(produccion_df, 'PC_BPPD', method='median')
produccion_df['WAT'] = impute_grouped_data(produccion_df, 'WAT', method='median')
produccion_df['GAS'] = impute_grouped_data(produccion_df, 'GAS', method='linear')


test_df['GRAV_OIL_API'] = impute_grouped_data(test_df, 'GRAV_OIL_API', method='linear')
test_df['GAS_SP_GRAVITY'] = impute_grouped_data(test_df, 'GAS_SP_GRAVITY', method='linear')

In [3]:
from sklearn.preprocessing import LabelEncoder

In [4]:
# Suponiendo que 'produccion_df' y 'test_df' ya están cargados y limpios

# Convertir las columnas de fecha al formato correcto
produccion_df['DIAS'] = pd.to_datetime(produccion_df['DIAS'], format='%d-%b-%y')
test_df['EFF_DT'] = pd.to_datetime(test_df['EFF_DT'])

# Combinar los DataFrames usando 'COMP_S_NAME' y las fechas correspondientes
combined_df = pd.merge(produccion_df, test_df, left_on=['COMP_S_NAME', 'DIAS'], right_on=['COMP_S_NAME', 'EFF_DT'], how='inner')

# Eliminar filas con valores faltantes en columnas críticas
critical_columns_produccion = ['DIAS', 'POZO', 'COMP_S_NAME']
critical_columns_test = ['BSW', 'CASING_PRESS', 'PBHP', 'TUBING_PRESS', 'PIP', 'FLAP', 'SBHP', 'TUBING_TMP', 'AMPS_A', 'PUMP_TMP', 'MOTOR_HZ', 'POWER_KW', 'STAGE_COUNT', 'VOLTAGE', 'INTAKE_DEPTH', 'PUMP_TYPE', 'SALINIDAD', 'INJECTION_VOL', 'RETURN_VOL']
combined_df.dropna(subset=critical_columns_produccion + critical_columns_test, inplace=True)

# Imputar valores faltantes en otras columnas con el método recomendado
# Asumiendo que la función 'impute_grouped_data' está definida anteriormente en el código
combined_df['PC_BPPD'] = impute_grouped_data(combined_df, 'PC_BPPD', method='median')
combined_df['WAT'] = impute_grouped_data(combined_df, 'WAT', method='median')
combined_df['GAS'] = impute_grouped_data(combined_df, 'GAS', method='linear')
combined_df['GRAV_OIL_API'] = impute_grouped_data(combined_df, 'GRAV_OIL_API', method='linear')
combined_df['GAS_SP_GRAVITY'] = impute_grouped_data(combined_df, 'GAS_SP_GRAVITY', method='linear')

# Extraer características de la columna de fecha 'DIAS'
combined_df['year'] = combined_df['DIAS'].dt.year
combined_df['month'] = combined_df['DIAS'].dt.month
combined_df['day'] = combined_df['DIAS'].dt.day
combined_df['dayofweek'] = combined_df['DIAS'].dt.dayofweek

# Crear un diccionario para almacenar los mapeos de LabelEncoder
le_dict = {}

# Aplicar Label Encoding a las columnas seleccionadas incluyendo 'COMP_S_NAME'
le = LabelEncoder()
columnas_produccion_label = ['POZO', 'COMP_S_NAME']
columnas_test_label = ['PUMP_TYPE']

# Aplicar Label Encoding a las columnas seleccionadas y guardar los mapeos
for columna in columnas_produccion_label + columnas_test_label:
    le = LabelEncoder()
    combined_df[columna] = le.fit_transform(combined_df[columna])
    # Guardar el objeto LabelEncoder en el diccionario
    le_dict[columna] = le

# Eliminar las columnas de fecha originales 'DIAS' y 'EFF_DT'
combined_df.drop(['DIAS', 'EFF_DT'], axis=1, inplace=True)

# Lista de variables predictoras incluyendo las nuevas características de fecha
predictoras = [
    'AMPS_A', 'TUBING_TMP', 'CASING_PRESS', 'PBHP', 'SALINIDAD',
    'GAS_SP_GRAVITY', 'STAGE_COUNT', 'POWER_KW', 'TUBING_PRESS',
    'PUMP_TMP', 'VOLTAGE', 'INTAKE_DEPTH', 'year', 'month', 'day', 'dayofweek',
    'POZO', 'COMP_S_NAME','PUMP_TYPE'
]

# Verificar si todas las columnas existen en el DataFrame
for columna in predictoras:
    if columna not in combined_df.columns:
        print(f"La columna {columna} no está en el DataFrame.")

# Definir las características (X) y las variables objetivo (y)
X = combined_df[predictoras]  # Variables predictoras
y = combined_df[['PC_BPPD', 'WAT', 'GAS']]  # Variables objetivo

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ahora X_train y y_train contienen los datos para entrenar el modelo
# X_test y y_test se usarán para probar el modelo

DISEÑO Y DESARROLLO DEL ARBOL

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Mejores hiperparámetros encontrados por GridSearchCV
mejores_hiperparametros = {
    'bootstrap': True,
    'max_depth': None,
    'min_samples_leaf': 1,
    'min_samples_split': 5,
    'n_estimators': 400     
}

# Inicializar el modelo Random Forest con los mejores hiperparámetros
mejor_rf = RandomForestRegressor(
    n_estimators=mejores_hiperparametros['n_estimators'],
    max_depth=mejores_hiperparametros['max_depth'],
    min_samples_split=mejores_hiperparametros['min_samples_split'],
    min_samples_leaf=mejores_hiperparametros['min_samples_leaf'],
    bootstrap=mejores_hiperparametros['bootstrap'],
    random_state=42
)

# Entrenar el modelo con los datos de entrenamiento
mejor_rf.fit(X_train, y_train)

# Evaluar el modelo con los datos de prueba
y_pred_mejor_rf = mejor_rf.predict(X_test)
mse_mejor_rf = mean_squared_error(y_test, y_pred_mejor_rf)
r2_mejor_rf = r2_score(y_test, y_pred_mejor_rf)

print(f'Mejor Random Forest MSE: {mse_mejor_rf}')
print(f'Mejor Random Forest R^2: {r2_mejor_rf}')

Mejor Random Forest MSE: 3564.295141078345
Mejor Random Forest R^2: 0.9739631904829306


In [6]:
import joblib

# Asumiendo que 'mejor_rf' es el modelo de Random Forest ya entrenado
# Guardar el modelo en la carpeta deseada
ruta_guardado = 'mejor_rf_modelo_entrenado1.pkl'
joblib.dump(mejor_rf, ruta_guardado)

print(f"Modelo guardado en {ruta_guardado}")

Modelo guardado en mejor_rf_modelo_entrenado1.pkl


In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Función para imputar la media o mediana agrupada por COMP_S_NAME
def impute_grouped_data(df, column, method='mean'):
    if method == 'median':
        return df.groupby('COMP_S_NAME')[column].transform(lambda x: x.fillna(x.median()))
    elif method == 'linear':
        return df[column].interpolate(method='linear')
    else:
        return df.groupby('COMP_S_NAME')[column].transform(lambda x: x.fillna(x.mean()))

# Cargar las bases de datos desde Excel para predicciones
prod_pred_df = pd.read_excel("Base enero 2024.xlsx", sheet_name='PROD')
test_pred_df = pd.read_excel("Base enero 2024.xlsx", sheet_name='PRUEBAS')

# Eliminar duplicados
prod_pred_df = prod_pred_df.drop_duplicates()
test_pred_df = test_pred_df.drop_duplicates()

# Convertir las columnas de fecha al formato correcto
prod_pred_df['DIAS_PRED'] = pd.to_datetime(prod_pred_df['DIAS'], format='%d-%b-%y')
test_pred_df['EFF_DT_PRED'] = pd.to_datetime(test_pred_df['EFF_DT'])

# Combinar los DataFrames usando 'COMP_S_NAME' y las fechas correspondientes
combined_pred_df = pd.merge(prod_pred_df, test_pred_df, left_on=['COMP_S_NAME', 'DIAS_PRED'], right_on=['COMP_S_NAME', 'EFF_DT_PRED'], how='inner')

# Filtrar la columna COMP_S_NAME por los valores específicos
valores_filtrados = ['SCH-099TI','SCHAP-482TI','SCHAC-353HI','SCHAO-475HUI','SCHD-251UI']
combined_pred_df = combined_pred_df[combined_pred_df['COMP_S_NAME'].isin(valores_filtrados)]

# Eliminar filas con valores faltantes en columnas críticas
critical_columns_prod_pred = ['ANIO', 'MES', 'DIAS_PRED', 'POZO', 'COMP_S_NAME']  # 'ROUTE_NAME' y 'PAD' no están disponibles
critical_columns_test_pred = ['BSW', 'CASING_PRESS', 'PBHP', 'TUBING_PRESS', 'PIP', 'FLAP', 'SBHP', 'TUBING_TMP', 'AMPS_A', 'PUMP_TMP', 'MOTOR_HZ', 'POWER_KW', 'STAGE_COUNT', 'VOLTAGE', 'INTAKE_DEPTH', 'PUMP_TYPE', 'SALINIDAD', 'INJECTION_VOL', 'RETURN_VOL']
combined_pred_df.dropna(subset=critical_columns_prod_pred + critical_columns_test_pred, inplace=True)

# Imputar valores faltantes en otras columnas con el método recomendado
combined_pred_df['PC_BPPD'] = impute_grouped_data(combined_pred_df, 'PC_BPPD', method='median')
combined_pred_df['WAT'] = impute_grouped_data(combined_pred_df, 'WAT', method='median')
combined_pred_df['GAS'] = impute_grouped_data(combined_pred_df, 'GAS', method='linear')
combined_pred_df['GRAV_OIL_API'] = impute_grouped_data(combined_pred_df, 'GRAV_OIL_API', method='linear')
combined_pred_df['GAS_SP_GRAVITY'] = impute_grouped_data(combined_pred_df, 'GAS_SP_GRAVITY', method='linear')

# Extraer características de la columna de fecha 'DIAS_PRED'
combined_pred_df['year'] = combined_pred_df['DIAS_PRED'].dt.year
combined_pred_df['month'] = combined_pred_df['DIAS_PRED'].dt.month
combined_pred_df['day'] = combined_pred_df['DIAS_PRED'].dt.day
combined_pred_df['dayofweek'] = combined_pred_df['DIAS_PRED'].dt.dayofweek


# Aplicar Label Encoding a las columnas seleccionadas incluyendo 'COMP_S_NAME'
columnas_prod_pred_label = ['POZO', 'COMP_S_NAME']  # 'ROUTE_NAME' y 'PAD' no están disponibles
columnas_test_pred_label = ['PUMP_TYPE']

# Aplicar Label Encoding a las columnas categóricas usando los mapeos existentes
for columna in columnas_prod_pred_label + columnas_test_pred_label:
    if columna in le_dict:
        le = le_dict[columna]
        # Transformar los datos usando el LabelEncoder existente
        combined_pred_df[columna] = le.transform(combined_pred_df[columna])
    else:
        print(f"La columna {columna} no fue codificada durante el entrenamiento y no se puede transformar.")

# Eliminar las columnas de fecha originales 'DIAS_PRED' y 'EFF_DT_PRED'
combined_pred_df.drop(['DIAS_PRED', 'EFF_DT_PRED'], axis=1, inplace=True)

# Ahora 'combined_pred_df' está listo para ser utilizado para hacer predicciones

In [10]:
pd.DataFrame(y_pred_mejor_rf).describe()

,0,1,2
count,2226.000000,2226.000000,2226.000000
mean,270.671506,330.616115,82.956867
std,228.139333,607.143259,116.723913
min,7.563509,2.026040,0.319247
25%,126.605145,23.139065,25.855027
50%,204.746882,81.675095,46.201392
75%,313.719675,324.033154,82.841809
max,1609.044769,4231.698896,807.087183


In [8]:
import pandas as pd
import joblib

# Cargar el modelo entrenado
mejor_rf = joblib.load('mejor_rf_modelo_entrenado1.pkl')

# Asegúrarse de que 'combined_pred_df' es el DataFrame preparado para la predicción
# ...

# Lista de variables predictoras basada en el modelo entrenado
predictoras_pred = [
    'AMPS_A', 'TUBING_TMP', 'CASING_PRESS', 'PBHP', 'SALINIDAD',
    'GAS_SP_GRAVITY', 'STAGE_COUNT', 'POWER_KW', 'TUBING_PRESS',
    'PUMP_TMP', 'VOLTAGE', 'INTAKE_DEPTH', 'year', 'month', 'day', 'dayofweek',
    'POZO', 'COMP_S_NAME','PUMP_TYPE'
]

# Preparar los datos para la predicción
X_pred = combined_pred_df[predictoras_pred]

# Realizar las predicciones con el modelo
predicciones = mejor_rf.predict(X_pred)

# Convertir las predicciones a un DataFrame y añadir las columnas de predicciones al DataFrame original
combined_pred_df['PC_BPPD_pred'] = predicciones[:, 0]  # Asumiendo que esta es la primera variable objetivo
combined_pred_df['WAT_pred'] = predicciones[:, 1]      # Asumiendo que esta es la segunda variable objetivo
combined_pred_df['GAS_pred'] = predicciones[:, 2]      # Asumiendo que esta es la tercera variable objetivo

# Revertir el Label Encoding para las columnas categóricas
for columna in ['POZO', 'COMP_S_NAME', 'PUMP_TYPE']:
    if columna in le_dict:
        le = le_dict[columna]
        combined_pred_df[columna] = le.inverse_transform(combined_pred_df[columna])

# Guardar los resultados en un nuevo archivo Excel
combined_pred_df.to_excel("Resultados_Predicciones.xlsx", index=False)